# Expenses analysis

## Overview

This Jupyter notebook, using F# as code cells language, is meant to analyse expenses provided in a CSV file at the location `/notebooks/expenses.csv`. I run it with  [IFSharp](https://github.com/fsprojects/IfSharp) in a docker container. [Having built the docker image](https://github.com/fsprojects/IfSharp#running-inside-a-docker-container) and named it `ifsharp`, I mount my local directory holding the expenses.csv file in the container with the command 
```
docker run -v /my/path/to/Documents/ipython:/notebooks -p 8888:8888 ifsharp
```

The CSV file format handled is the one exported from the Android application Hello Expense, but providing the same format from another app will work too. The lines of the csv file has to be:
```
"date","Category","amount","currenct","Note","CommaSeparatedTags"
```
For example
```
"21/1/2018","Other","38,72","€","ISP","Home,Internet"
```
## Parameters

All charts and computations are based on the list of expenses in `taggedExpenses`, which is build in the secion "Main Expenses List computation". Filtering expenses at that level will impact all charts in this notebook.
If specific parameters are available for a section, those are specified at the start of the section.

In [ ]:
#load "Paket.fsx"
Paket.Package ["XPlot.Plotly";"MathNet.Numerics"; "MathNet.Numerics.FSharp"; "FSharp.Data"]
#load "XPlot.Plotly.fsx"
#load "Paket.Generated.Refs.fsx"


open MathNet.Numerics
open System
open XPlot.Plotly
open FSharp.Data

let headers = ["date","category","amount","currency", "note","tags"]

"OK"

In [ ]:
(* Defining types *)


(* Change the culture in the next line if you use another float format than 1.245,45 (continental europe format) *)
type Expenses = CsvProvider<"/notebooks/expenses.csv",HasHeaders=false, Schema="date(date),category(string),amount(float),currency(string),note(string),tags(string)", Culture="DE">
let e = Expenses.Load("/notebooks/expenses.csv")
(* define a type for our expenses where tags are put in a list (and not one string containing comma spearated tags)  *)
type TaggedExpense = { Date : DateTime ; Category:  string ; Amount: double ; Currency: string ; Note: string; Tags: string list}


## Filters
These are example function that can be used to filter the `taggedExpenses` list.
Notes and tags available are listed below (sections "Existing Notes" and "Existing Tags").
To use these filters, just add a call at the end of the computation of `taggedExpenses`. Here is an example to keep only expenses that where tagged with `Travel`:
```
let taggedExpenses = e.Rows |>Seq.map(expensesTagger) |> Seq.toList |> List.filter (tagFinder "Travel")
```

In [ ]:
(* Filter functions
*)
let tagFinder (tag:string) (r:TaggedExpense) :bool = r.Tags |> List.contains tag
let noteFinder (note:string) (r:TaggedExpense) :bool = (r.Note=note)
let tagExcluder (tag:string) (r:TaggedExpense) :bool = not (r.Tags |> List.contains tag)
let totalAmount l = l|> List.fold(fun acc r -> acc+r.Amount) 0.
"OK"

## Main Expenses List computation

In [ ]:

(* convert tags string to tags list*)
let expensesTagger (r:Expenses.Row) :TaggedExpense = 
         let tagsSeparator = System.Text.RegularExpressions.Regex(@"\s*,\s*") in
         { Date=r.Date; 
           Category = r.Category; 
           Amount =  r.Amount; 
           Currency = r.Currency; 
           Note= r.Note; 
           Tags=r.Tags|> tagsSeparator.Split |> Seq.toList ;
         }

(* Sequence of rows with tags list*)
let taggedExpenses = e.Rows |>Seq.map(expensesTagger) |> Seq.toList 


let tags = taggedExpenses|> List.map(fun r -> r.Tags) |>  List.concat |> List.distinct |> List.sort
let notes = taggedExpenses|> List.map(fun r -> r.Note) |> List.distinct |> List.sort



"OK"


## Existing Notes

In [ ]:
notes |> Display

## Existing Tags

In [ ]:
tags |> Display


# Helper functions

In [ ]:
(* basic function to compute median value of an ordered list of floats *)
let median floats = let l = List.sort floats in 
                    let length = List.length l in 
                    if (length % 2) = 0  then ((List.item (length/2-1) l) + (List.item (length/2) l) )/2.
                    else float(List.item (length/2) l)

"OK"

(*
median [1.;2.;3.;4.;5.;6.]
*)


In [ ]:
(* These are functions related to the building of expenses groups summaries
*)

(* Record holding count, total amount, median amount and average amount of an expenses group
   Usually it will be for expenses with the same not, or expenses with the same tag.
   But it should also be usable for expenses grouped per day for example.
*)
type ExpensesSummary = 
    { Count: int; 
      Total: float; 
      Median: float   } 
    with member m.Average = m.Total / float(m.Count)

(* build Sequence of tuples (expensesGroupName:string,ExpensesSummary)
*)

let computeExpensesSummaries (l:(string*TaggedExpense list) list) =  l|> List.map (fun  (groupName,expenses)-> 
                                    (groupName, 
                                     { Count  = expenses|>List.length;
                                       Total  = expenses|>List.sumBy (fun e -> e.Amount);
                                       Median = expenses|>List.map (fun e -> e.Amount) |>median
                                     }
                                    )
                                  )

"OK"


In [ ]:
(* These are funtions related to bargraphs of expenses

*)


(* Function displaying a Bar graph of the total amount, with lines for average.
  s is of type s:seq<(string,ExpensesSummary)>
*)
let expensesGroupBarGraph (notesDisplayed:(string*ExpensesSummary) list) = 
(* Bar graph displaying notes occurences, with labels for total and average amounts *)
  let dailyBars = (Bar( x = (notesDisplayed
                         |> List.map (fun (note,_) -> note)) , 
                     y = (notesDisplayed
                         |> List.map (fun (_,{Total = v;}) -> v)),
                     text = (notesDisplayed
                         |> List.map (fun (s,r) -> 
                                     sprintf "%s Total: %.2f \n Avg: %.2f \n Median: %.2f \n Count: %i" s r.Total r.Average r.Median r.Count )),
                     marker = Marker(color = "rgb(105, 133, 180)")
                ) :> Trace)

(* Line graph to display average expense for each note *)
  let average =
    (Scatter(
        x = (notesDisplayed
                       |> List.map (fun (note,_) -> note)) ,
        y = (notesDisplayed
                       |> List.map (fun (_,r) -> r.Average)),
        name = "Average",
        mode = "lines+markers" ) :> Trace)

  let count =
    (Scatter(
        x = (notesDisplayed
                       |> List.map (fun (note,_) -> note)) ,
        y = (notesDisplayed
                       |> List.map (fun (_,r) -> r.Count)),
        name = "Count",
        line = Line(
                  color = "#77ff00",
                  width = 0.5
               ),
        mode = "lines+markers" ) :> Trace)

(* Display a logarithmic scale *)
  let layout =
    Layout(
        title = "Logarithmic Axes",
        yaxis =
            Yaxis(
                ``type`` = "log",
                autorange = true
            )
     )
  [dailyBars;average;count] |> Chart.Plot |> Chart.WithLayout layout




"OK"

## Expenses grouped by Note

### Parameters

In [ ]:
let minimumOccurencesForDisplay=3

### Code and chart

In [ ]:
(* Expenses grouped by note, with their summary built
*)

let notesExpenses = taggedExpenses 
                      |> List.groupBy (fun e -> e.Note) 
                      |> computeExpensesSummaries



(* retain only notes with required minimal occurences*)
let notesDisplayed = notesExpenses
                   |> List.filter (fun (_, {Count = count; }) -> count >= minimumOccurencesForDisplay) 
                   |> List.sortBy (fun (_, {Total = v;}) -> -v)
expensesGroupBarGraph notesDisplayed


## Expenses grouped by tag
### Parameter

In [ ]:
let minimumOccurencesForDisplay=3

In [ ]:
(* Expenses per tag *)

(* add `expense` to `acc` *)
let rec addTag (acc:(string*TaggedExpense list) list) (expense:TaggedExpense) (tag:string) = 
  match acc with
(*
- is is possible to only match on Tag and still have the whole record put in a variable to be used in the body of the match?
*)  
  (* in the case the tag we want is in the head of the list, we add the expense the the head's expenses list
     We can then stop the recursion as we have found the element to update
  *)
  | (currentTag,expenses) :: tail when currentTag = tag -> (currentTag, expense::expenses) ::tail
  (* in this case the first element of acc does not match the tag we want, so we do the recursive call*)
  | head :: tail -> head :: addTag tail expense tag
  (* tag was not found in acc, so we add it at the end of the list*)
  | [] -> ( tag, [ expense ]) :: []


(* handle all tags for one expense
*)
let rec addTagsExpense (expense:TaggedExpense) (tags:string list) (acc:(string*TaggedExpense list) list) = 
  match tags with
  | tag :: tail -> addTag acc expense tag |> addTagsExpense expense tail
  | []  -> acc

(* acc groups expenses per tag. It is of the form [(tag1, [10.;3.;45.]); (tag2, [3.;4.;76.;6.30]) .... ]
   Expenses are recorded under multiple elements of add if it was assigned multiple tags
*)
let rec tagsExpenses  (l:TaggedExpense list) (acc:(string*TaggedExpense list) list) :((string*TaggedExpense list) list) = 
  match l with
  (* We add the head of the list to acc, and the result is passed as the acc value for the recursive call on tail
  *)
  | head :: tail -> addTagsExpense head head.Tags acc |> tagsExpenses tail
  (* When at the end of the list, return acc *)
  | [] -> acc 


let expensesPerTag = tagsExpenses ( taggedExpenses |> Seq.toList) [] 

"OK"

In [ ]:
(* Expenses grouped by note, with their summary built
*)
let tagsExpenses = expensesPerTag 
                      |> computeExpensesSummaries



(* retain only notes with required minimal occurences*)
let tagsDisplayed = tagsExpenses
                   |> List.filter (fun (_, {Count = count; }) -> count >= minimumOccurencesForDisplay) 
                   |> List.sortBy (fun (_, {Total = v;}) -> -v)
expensesGroupBarGraph tagsDisplayed


## Daily expenses

Here is a chart representing the total daily expense vs date.

### Parameters
You can filter the expenses displayed in the chart with the filter functions that were defined at the start of the notebook (Section Filter).
Add calls to these functions in the pipe computing the value of `dailyExpenses`, which is based initially on `taggedExpenses`.
You can also use a logarithmic scale by uncommenting the line
```
    ``type`` = "log",
```
in the code below.


In [ ]:
(* Add filters in the next expression to change which expenses are displayed in the chart below.
*)
let dailyExpenses = taggedExpenses 
(*
                    |> List.filter (tagFinder "IncludeTag") 
                    |> List.filter (tagExcluder "ExcludeTag")
*)
                    |> List.groupBy (fun { Date = it } -> it) 
                    |> List.map (fun (date, it) ->
                                                (date, (it |> List.sumBy (fun { Amount = it } -> it)))
)

let dates = dailyExpenses |> List.map (fun (date, _) -> date)
let expenses = dailyExpenses |> List.map (fun (_, expense) -> expense)

let dailyBars = Bar( x=dates, y=expenses)
let layout =
    Layout(
        title = "Daily Expenses",
        yaxis =
            Yaxis(
(*                ``type`` = "log", *)
                autorange = true
            )
    )
dailyBars |> Chart.Plot |> Chart.WithLayout layout


## Daily Expenses Moving Average
When using a moving average, the value of a day is the average over p the previous days, where p is the moving average period. This is super useful if you travel and have some expenses that cover multiple days, eg if you stay in an hotel for a couple of days. It will spread that expense over the moving average period you give.

### Parameters
You can set the moving average period with the variable `movingAveragePeriod`.
You can also use a logarithmic scale by uncommenting the line
```
    ``type`` = "log",
```
in the code below.

In [ ]:
(* You can set a moving average perdiod. 
*)
let movingAveragePeriod = 7

In [ ]:
(* from http://www.fssnip.net/4S/title/Moving-Average , not checked *)
let movingAverage (period : int) (values : float seq) =
    Seq.zip values (Seq.skip period values)
    |> Seq.scan (fun last (prev, cur) -> last - prev + cur) (values |> Seq.take period |> Seq.sum)
    |> Seq.map (fun x -> x / float(period))

(*
[3.;4.;5.;7.;3.;8.;5.;9.;1.;5.;3.] |> movingAverage 2
*)

"OK"

In [ ]:


let movingAverage (period : int) (values : float seq) =
    Seq.zip values (Seq.skip period values)
    |> Seq.scan (fun last (prev, cur) -> last - prev + cur) (values |> Seq.take period |> Seq.sum)
    |> Seq.map (fun x -> x / float(period))



let dates = dailyExpenses |> List.map (fun (date, _) -> date)
let expenses = dailyExpenses |> List.map (fun (_, expense) -> expense) |> List.toSeq |> movingAverage movingAveragePeriod |> Seq.toList

let dailyBars = Bar( x=dates, y=expenses)
let layout =
    Layout(
        title = sprintf "Daily Expenses, Moving Average (%d days period)" movingAveragePeriod,
        yaxis =
            Yaxis(
(*                ``type`` = "log", *)
                autorange = true
            )
    )
dailyBars |> Chart.Plot |> Chart.WithLayout layout


# Expenses Visualisation

Below is a scatter plot of expense amount vs date.
This possibly will let you identify what king of expenses are most common.

In [ ]:
let b = Scatter( x=(taggedExpenses|> List.map(fun r -> r.Date)), 
                 y=(taggedExpenses|> List.map(fun r -> r.Amount)), 
                 mode = "markers",
                 marker=Marker(
                  color = "rgb(142, 124, 195)",
                  size = 5
                 )
        )

let styledLayout =
    Layout(
        title = "All Expenses Scatter Plot",
        xaxis =
            Xaxis(
                title = "Date",
                showgrid = true,
                zeroline = false
            ),
        yaxis =
            Yaxis(
                title = "€"
(*                ``type`` = "log" *)
            )
    )
b |> Chart.Plot |> Chart.WithLayout styledLayout